In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import xarray as xr

In [3]:
%run create_JASADCP_metadata_df.ipynb

['3 deg (???)' 'not specified' 'unconfirmed (narrow band)' 'not listed'
 'unknown:' '30 degrees']
22


In [223]:
df.columns

Index(['fname', 'hardware_model', 'instrument_name', 'serial_numbers',
       'transmit_frequency', 'phased_array', 'cruise_beg_date',
       'blanking_interval', 'bin_length', 'transducer_beam_angle',
       'transmit_pulse_length', 'comments', 'biomass_dtmn', 'geo_region',
       'chief_scientist', 'personnel_charge', 'manufacturer',
       'transducer_config', 'depth_range', 'num_bins', 'ens_avg_intvl',
       'ss_calc', 'project', 'nb150', 'bandwidth'],
      dtype='object')

In [254]:
#'VM-150', 'RD-VM150', 'VM150', 'RD-VM0150'
#'VM-300', 'RD-VM300'
hwmodelnow = 'RD-VM150'
print(len(df[(df['hardware_model']==hwmodelnow)
             & (df['nb150'].notnull())
             & (df['geo_region'].str.contains('acific'))]))
print(len(df[(df['hardware_model']==hwmodelnow) & (df['nb150'].notnull())]))
print(len(df[(df['hardware_model']==hwmodelnow)]))
df[(df['hardware_model']==hwmodelnow) & (df['nb150'].notnull())]

2
2
166


,fname,hardware_model,instrument_name,serial_numbers,transmit_frequency,phased_array,cruise_beg_date,blanking_interval,bin_length,transducer_beam_angle,...,personnel_charge,manufacturer,transducer_config,depth_range,num_bins,ens_avg_intvl,ss_calc,project,nb150,bandwidth
753,00803_short.nc,RD-VM150,VM-150,None,None,None,2004-06-17 00:23:57,4m,8m,None,...,Jules Hummon,RD Instruments (RDI),None,17m to 571m (bin centers),70,300 seconds,FUNCTION OF TEMP AT TRANSDUCER,CLIVAR Global Carbon Program;repeat of WOCE One-time Line,nb150///NB150///nb150///nb150,None
755,00805_short.nc,RD-VM150,VM-150,None,None,None,2004-07-25 07:32:07,4m,8m,None,...,Jules Hummon,RD Instruments (RDI),None,17m to 571m (bin centers),70,300 seconds,FUNCTION OF TEMP AT TRANSDUCER,CLIVAR Global Carbon Program;repeat of WOCE One-time Line,nb150///NB150,None


In [249]:
# - Look at all VM-150 w/ something about nb150 in metadata
df[(df['hardware_model']=='VM-150') & (df['nb150'].notnull()) &
   (df['geo_region'].str.contains('acific'))
  ][['fname','cruise_beg_date','geo_region','project','bandwidth','comments','nb150']]
print(len(df[(df['hardware_model']=='VM-150')
             & (df['nb150'].notnull())
             & (df['geo_region'].str.contains('acific'))]))
# --> All 10 of these are confirmed to be NB!
print(len(df[(df['hardware_model']=='VM-150') & (df['nb150'].notnull())]))
# --> I also checked 1 Southern Ocean one and it was confirmed to be NB
# --> Let's just assume all 130 w/ something about nb150 in metadata==NB
print(len(df[(df['hardware_model']=='VM-150')]))

10
130
578


In [ ]:
df[(df['hardware_model']=='VM-150')][['fname','cruise_beg_date','geo_region','project','bandwidth','comments','nb150']]

In [ ]:
df[(df['hardware_model']=='VM-150') & (df['project']=='Hawaii Ocean Time-series (JGOFS, WOCE (time-series))')]['fname'].values

In [ ]:
df[df['hardware_model']=='VM-150']['fname'].values

In [ ]:
# - Figuring out whether VM-150 is BB or NB (1996 might be the cutoff)
#00039-00050 --> ? 1988-1991, HOT cruises, doesn't even say here: http://hahana.soest.hawaii.edu/hot/reports/rep_y1.pdf
#00057-00077 --> ? 1991-1994, HOT cruises, doesn't even say here: http://hahana.soest.hawaii.edu/hot/reports/rep_y3.pdf, to y6.pdf
#00080-00082 --> ? 1994
#00096-00106 --> ? 1994-1995, HOT cruises
#00124, 00134, 00135 --> ? 1996
#00136 --> ? 1995
#00137 --> ? 1995
#...didn't check
#01695 --> NB from CRUISE_NAME, COMMENTS, ATTACHMENT TO THE HULL
#...didn't check
#02180 --> NB from COMMENTS 

In [216]:
#'road' in strnow
re.findall('band', strnow, re.IGNORECASE)

[]

In [8]:
ncfile = '/opt/acoustic-variability/data/JASADCP/ncfiles/00328_short.nc'
ncnow = xr.open_dataset(ncfile)
strnow = ncnow.attrs['cruise_sonar_summary']
ncnow.attrs['cruise_sonar_summary'].split('\n')

[' #DATA_DATES: 1998/05/24 13:59:00 --- to --- 1998/06/05 07:22:00',
 ' #LON_RANGE: 124.59 W --- to --- 121.86 W',
 ' #LAT_RANGE:  36.55 N --- to ---  39.18 N',
 ' #DEPTH_RANGE:     16 --- to --- 488 m',
 ' #SAC_CRUISE_ID: 00328 ',
 ' #PLATFORM_NAME: R/V David Starr Jordan ',
 ' #PRINCIPAL_INVESTIGATOR_NAME: Alec MacCall and Steve Ralston',
 ' #PI_INSTITUTION: National Marine Fisheries Service, Tiburon Lab ',
 ' #PI_COUNTRY: USA ',
 ' #PROJECT: Rockfish Recruitment Survey',
 ' #CRUISE_NAME: DSJ 9807 Leg 2 ',
 ' #PORTS: San Francisco to San Francisco ',
 ' #GEOGRAPHIC_REGION: Central California Coastal Waters',
 ' #PROCESSED_BY: NMFS, Tiburon Lab',
 ' #NAVIGATION: Differential GPS',
 ' #QUALITY_NAV: good',
 ' #GENERAL_INFORMATION: ',
 'CRUISE NOTES',
 '  CHIEF SCIENTIST ON SHIP       :Dave Woodbury',
 '    INSTITUTE                   :NMFS Tiburon Lab',
 '    COUNTRY                     :USA',
 '  SIGNIFICANT DATA GAPS         :NONE',
 '  SPECIAL SHIP TRACK PATTERNS   :',
 '  COMMENTS  

# Further questions:
- Are the Kc and Er values in Jerry Mullison's email from phase II or phase I OS?
- Where does the speed of sound c in the denominator go comparing 1998 TRDI field service note vs. Mullison 2017? What c should I use for the denominator, 1478.1 m/s like for calculating the depth cells in 1998 TRDI field service note??
- So I can just use the lowest measured E per cruise to define Er? Some papers do this, cite them. Or if I know that all cruises used the same ADCP, I could find the lowest measured E out of all of those cruises. (Er should be constant for a given ADCP according to Mullison 2017.)
- What year did TRDI switch from OS to OS-II?
- If the angle from the vertical is missing, can I assume 30 degrees? This is true for all configs? What are the usual/standard thetas for each instrument type?
- What years were all these instruments available? Can I potentially use cruise year to figure what instrument was being used? Diagram of evolution of ADCP models? Or years when you were supporting them?
- Send TRDI a spreadsheet of all VM-150, VM-300 to see if they can help figure out which is BB and which is NB.
- Get mis-timed cruises from Peter Gaube.
- Is direct-reading (DR) meaning that it's moored? (can try to check website)
- If blank distance/blanking interval (or whatever it's called) is missing, can I assume it's the same as bin length (or transmit pulse length maybe)? Other specific calculation questions like this.
- WH300 had only bb or nb before some time?
- VM150 and/or VM300 had only bb or nb before some time?

# Quick stats from create_JASADCP_instrument_spreadsheet.ipynb df

**All names to become 'VM-150'**:\
'VM-150', 'RD-VM150', 'VM150', 'RD-VM0150'

**All names to become 'VM-300'**:\
'VM-300', 'RD-VM300'

**All names to become 'UNCLEAR'**:\
'150' (1990-1998; assume RDI + VM + NB or BB),  
'RDI' (1985-1993; some give you transmit frequency, w/ those just assume VM + either NB or BB),  
'150 kHz hull mounted ADCP' (1991; assume RDI + VM + NB or BB),  
'Narrowband' (1994-1996; all 153.6kHz transmit frequency; assume RDI + VM + NB),  
'150 kHz' (1994; assume RDI + VM + NB or BB),  
'153.6 kHz hull mounted ADCP' (2002; assume RDI + maybe OS?)

In [125]:
#pd.set_option('max_rows', 1000)
#pd.set_option('display.max_colwidth', -1)

# - fraction of cruises w/ weird names
weird_names = ['VM-150', 'RD-VM150', 'VM150', 'RD-VM0150', 'VM-300', 'RD-VM300', '150',
               'RDI', '150 kHz hull mounted ADCP', 'Narrowband', '150 kHz', '153.6 kHz hull mounted ADCP'] 
weird_names_vm = ['VM-150', 'RD-VM150', 'VM150', 'RD-VM0150', 'VM-300', 'RD-VM300']
print(len(df[df['hardware_model'].isin(weird_names)])/len(df)) # --> 0.3580301685891748
print(len(df[df['hardware_model'].isin(weird_names_vm)])/len(df)) # --> 0.34072759538598046
print(len(df[df['hardware_model']=='VM-150'])/len(df)) # --> 0.25643300798580304
print(len(df[df['hardware_model']=='RD-VM150'])/len(df)) # --> 0.07364685004436557
print(len(df[df['hardware_model']=='VM150'])/len(df)) # --> 0.0008873114463176575
print(len(df[df['hardware_model']=='RD-VM0150'])/len(df)) # --> 0.00044365572315882877
print(len(df[df['hardware_model']=='VM-300'])/len(df)) # --> 0.005767524401064774
print(len(df[df['hardware_model']=='RD-VM300'])/len(df)) # --> 0.00354924578527063

# - fraction of cruises w/ weird names in Pacific
print(len(df[(df['hardware_model'].isin(weird_names)) & (
    df['geo_region'].str.contains('acific') |
    df['geo_region'].str.contains('alifornia'))])/len(df)) # --> 0.17701863354037267
print(len(df[(df['hardware_model'].isin(weird_names_vm)) & (
    df['geo_region'].str.contains('acific') |
    df['geo_region'].str.contains('alifornia'))])/len(df)) # --> 0.17213842058562556

# - fraction of cruises in Pacific
print(len(df[df['geo_region'].str.contains('acific') |
             df['geo_region'].str.contains('alifornia')])/len(df)) # --> 0.6313220940550133

# - earliest and latest cruises using OS 
print(df[(df['instrument_name'].str.contains('OS-'))]['cruise_beg_date'].min()) # --> '2000-08-31 00:01:35'
print(df[(df['instrument_name'].str.contains('OS-'))]['cruise_beg_date'].max()) # --> '2018-11-15 17:22:09'

# - earliest and latest cruises using NB-VM
print(df[(df['instrument_name'].str.contains('VM')) &
         (df['bandwidth']=='narrowband')]['cruise_beg_date'].min()) # --> '1989-08-15 10:20:00'
print(df[(df['instrument_name'].str.contains('VM')) &
         (df['bandwidth']=='narrowband')]['cruise_beg_date'].max()) # --> '2014-03-20 10:06:28'

# - earliest and latest cruises using BB
print(df[(df['hardware_model'].str.contains('road')) &
         ~(df['instrument_name'].str.contains('OS'))]['cruise_beg_date'].min()) # --> '1996-07-02 04:42:00'
print(df[(df['hardware_model'].str.contains('road')) &
         ~(df['instrument_name'].str.contains('OS'))]['cruise_beg_date'].max() # --> '2014-08-01 12:11:31'
# --> It could be that cruises that ONLY say VM and not BB or NB are NB and happened BEFORE BB existed

# - justification that VM usually means narrowband (though doesn't really work,
# given that 25.6% of all cruises are VM-150 w/ unknown BB or NB)
print(len(df[(df['instrument_name'].str.contains('VM-')) & (df['bandwidth']=='broadband')]['bandwidth'])) # --> 0
print(len(df[(df['instrument_name'].str.contains('VM-')) & (df['bandwidth']=='narrowband')]['bandwidth'])) # --> 178
print(len(df[(df['hardware_model'].str.contains('VM')) & (df['bandwidth']=='broadband')]['bandwidth'])) # --> 0
print(len(df[(df['hardware_model'].str.contains('VM')) & (df['bandwidth']=='narrowband')]['bandwidth'])) # --> 178

0.3580301685891748
0.34072759538598046
0.17701863354037267
0.17213842058562556
0.6313220940550133
2000-08-31 00:01:35
2018-11-15 17:22:09
0
178
0
178


In [ ]:
# - Look at broadband instruments
#df[df['bandwidth']=='broadband'] # --> too hard to look at

# - Look at only specific OS-75 cruises where they separated broadband and narrowband measurements
#dfnow = df.dropna(subset=['comments'])
#dfnow[dfnow['comments'].str.contains('separate set')]
# --> looks like broadband/narrowband always has bin_length, transmit_pulse_length, depth_range, num_bins, ens_avg_intvl
# as 8m/16m, 8m/16m, 25-657m/32-1136m, 80/70, 300s/300s

# - Look at instruments w/ broadband in hardware_model
#df[(df['hardware_model'].str.contains('road'))]
#df[(df['hardware_model'].str.contains('road')) & (df.index>1367)]
# --> ignore OS for now
# --> BB-150: blank = 4, bin = 8, range = 17-413/16-408, numbins = 50, ens = 300/60/180
# --> BB-150 concave: blank = 5, bin = 8, range = 30-332, numbins = 40, ens = 60

# - Look at narrowband instruments
#df[(df['instrument_name'].str.contains('VM')) & (df['bandwidth']=='narrowband')]
# --> VM-150 (NB): blank = 4, bin = 8, range = 400, numbins = 60, ens = 3min
# --> lots more, but doesn't seem any different from BB :(

# Calculate Sv for each ncfile

#### Mean volume backscattering strength eqn (Mullison 2017):  
$S_v = C + 10log((T_x + 273.16)*R^2) - L_{DBM} - P_{DBW} + 2 \alpha R + 10log(10^{k_c(E-E_r)/10} - 1)$, where:  
- C = constant combining several params specific to each instrument
- $T_x$ = temperature measured at the transducer ($^{\circ}$C)
- R = along-beam range to the measurement, taken in the last quarter of the bin for Workhorse, Long Ranger, and Quartermaster, and at midpt of the bin for other instruments
- $L_{DBM}$ = 10log(transmit pulse length, meters)
- $P_{DBW}$ = 10log(transmit power, Watts)
- $\alpha$ = absorption coefficient of the water
- $k_c$ = RSSI slope (dB/count)
- $E_r$ = noise floor (counts)

#### Mean volume backscattering strength eqn (fst-003):  
$S_v = 10log(\frac{4.47 \times 10^{-20} K_2 K_S}{c}) + 10log((T_x + 273.16)*R^2) - 10log(P) - 10log(K_1) + 2 \alpha R + 10log(10^{k_c(E-E_r)/10} - 1)$
- $K_2$ = system noise factor (dimensionless)
- $K_S$ = system constant, depends on NBADCP frequency
- c = speed of sound at the scattering layer being measured
- P = transmit pulse length
- $K_1$ = real-time power into the water (Watts) 
- All others the same as Mullison 2017 eqn

#### NBADCP-specific values and calculations (fst-003):

$C$:  
$C = 10log[\frac{4.47 x 10^{-20} K_2 K_s}{c}]$

$K_2$:

In [18]:
# From fst-003, pg 6:
arrays = [['VM', 'VM', 'VM', 'VM', 'VM', 'DR', 'DR', 'DR', 'DR', 'DR'],
          [75, 150, 300, 600, 1200, 75, 150, 300, 600, 1200]]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names = ['NBADCP Model', 'Frequency (kHz)'])
K2_values = pd.DataFrame([2.5, 4.3, 4.5, 9.1, 10.5, 2.2, 3.6, 4.2, 7.1, 8.1],index=index,columns=['K2'])
df_K2_values = pd.DataFrame(K2_values)
df_K2_values

K2
NBADCP Model Frequency (kHz)      
VM           75                2.5
             150               4.3
             300               4.5
             600               9.1
             1200             10.5
DR           75                2.2
             150               3.6
             300               4.2
             600               7.1
             1200              8.1

$K_S$:

In [11]:
# From fst-003, pg 10:
Ks_values = {
    'NBADCP frequency (kHz)' : [75, 150, 300, 600, 1200],
    'Ks' : [1.09E5, 4.17E5, 7.69E5, 1.56E6, 5.65E6]
}
df_Ks_values = pd.DataFrame(Ks_values)
df_Ks_values

,NBADCP frequency (kHz),Ks
0,75,109000.0
1,150,417000.0
2,300,769000.0
3,600,1560000.0
4,1200,5650000.0


$K_1$:

$K_1 = [\frac{(V_s \times a)-b}{c}]^2 \times K_{1c}$

$K_{1c}$:

In [11]:
# From fst-003, pg 10:
Ks_values = {
    'NBADCP frequency (kHz)' : [75, 150, 300, 600, 1200],
    'Ks' : [1.09E5, 4.17E5, 7.69E5, 1.56E6, 5.65E6]
}
df_Ks_values = pd.DataFrame(Ks_values)
df_Ks_values

,NBADCP frequency (kHz),Ks
0,75,109000.0
1,150,417000.0
2,300,769000.0
3,600,1560000.0
4,1200,5650000.0


$R$:

$R = \frac{B + |(P-D)/2| + (N \times D) + (D/4)}{cos(\theta)} \times \frac{c'}{1475.1}$

$K_c$:

For E < 200 counts, $K_c = \frac{127.3}{T_e+271}$  
If E is 200-230 counts, calibration of $K_c$ must be done at RDI. If E > 230 counts, $K_c$ can't be calibrated.  
$T_e$ = "temperature of system electronics - $T_e$ is used to calculate $K_c$ and $E_r$. $E_r$...is particularly sensitive to changes in $T_e$, so it is crucial to obtain an accurate record of $T_e$."  
"For DR-NBADCPs in which both system electronics and transducer assembly are immersed, $T_x$ can be substituted for $T_e$."  
"The ambient temp for the sm electronics will generally differ from the temp recorded at the transducer. Therefore, you must independently measure and record the temperature of the system electronics."

$E_r$:

#### For each ncfile, save the following variables along with Sv:
- Tx (temp at transducer, dims = time, units = degC)
- R (range, NBADCP samples in the last quarter of each depth cell/OS at midpt, dims = depth, units = m)
- Kc (RSSI slope, constant, units = dB/count)
- E (RSSI, dims = depth x time, units = counts)
- Er (noise floor, constant, units = counts) 
- P (transmit pulse length, 10log(P) = LDBM, constant, units = m)
- K1 (assumed or measured transmit power, 10log(K1) = PDBW, dims = time, units = Watts)
- alpha (attenuation coefficient)
- c (speed of sound, dims = depth x time, units = m/s)
- theta (angle of the transducer beams to vertical, constant, units = degrees)
- D (depth cell length, constant, units = m)
- Ks (if applicable - only for NBADCP?)
- K2 (if applicable - only for NBADCP?)
- K1c (if applicable - only for NBADCP?)
- B (blank beyond transmit, if applicable - only for NBADCP?, constant, units = m)

Metadata should include:
- Instrument type
- Instrument SN
- Transmit frequency

### Narrow df down to Pacific region, OS instruments for now

In [5]:
print(df['geo_region'].isnull().sum())
print(len(df))
dfnow = df.dropna(subset=['geo_region'])
print(len(dfnow))
dfnow = dfnow[dfnow['geo_region'].str.contains('acific')]
print(len(dfnow))
dfnow = dfnow[dfnow['instrument_name'].str.contains('OS')]
print(len(dfnow))
dfnow = dfnow[(dfnow['instrument_name']!='OS-II-38')&(dfnow['instrument_name']!='OS-II-75')] 

33
2254
2221
1274
611


## Load table of C, PDBW values

In [ ]:
cpdbw = pd.read_csv(dpath + 'typical_system_characteristics_table2_mullison2017.csv')

In [9]:
cpdbw.head()

,Instrument,C (25%) (dB),C (6%) (dB),P_DBW Battery (dB),P_DBW Power Supply (dB),Rayleigh Distance (m)
0,ChannelMaster 300,-143.44,-152.26,NaN,15.1,2.69
1,ChannelMaster 600,-139.08,-147.28,NaN,12.0,2.96
2,ChannelMaster 1200,-127.13,-137.17,NaN,9.0,1.71
3,Explorer Phased Array,NaN,-139.14,NaN,9.0,1.67
4,Explorer Piston,-132.73,-140.95,NaN,3.0,1.35


## Load table of kc, Er values

#### Nominal Kc values from TRDI field service email:
BBADCP: 0.45 dB/count  
WHADCP: 0.42 dB/count  
OS-II-75, SN1508: 0.373 (beam 1), 0.386 (beam 2), 0.388 (beam 3), 0.384 (beam 4) = 0.38275 (avg)  
OS-II-75, SN10656: 0.398 (beam 1), 0.389 (beam 2), 0.3988 (beam 3), 0.395 (beam 4) = 0.395 (avg)  
#### Nominal Kc values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 0.41-0.44, 0.42, 0.012   
OS75: 0.36-0.42, 0.39, 0.019  
OS38: 0.36-0.37, 0.37, 0.004  

#### Nominal Er values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 18-28, 22, 2.08  
OS75: 11-25, 19, 2.93  
OS38: 5-33, 14, 4.83  

In [10]:
# - Calc avg Kc for OS-II w/ SNs
print(np.mean([0.373, 0.386, 0.388, 0.384]))
print(np.mean([0.398, 0.389, 0.398, 0.395]))

0.38275000000000003
0.395


## Load WOD data (T,S)

## Compute Sv

In [20]:
dfnow.iloc[0]

fname                                                       00804_short.nc
hardware_model                                           Ocean Surveyer 75
instrument_name                                                      OS-75
serial_numbers                                                        None
transmit_frequency                                                    None
phased_array                                                          None
cruise_beg_date                                        2004-06-15 06:33:10
blanking_interval                                                       8m
bin_length                                                              8m
transducer_beam_angle                                                 None
transmit_pulse_length                                                   8m
comments                 narrowband mode///mounted with the ADCP pointi...
biomass_dtmn                                                            no
geo_region               

Mean volume backscattering strength:  
$S_v = C + 10log((T_x + 273.16)*R^2) - L_{DBM} - P_{DBW} + 2 \alpha R + 10log(10^{k_c(E-E_r)/10} - 1)$

In [ ]:
ncfile = dpath + 'JASADCP/ncfiles/00001_short.nc'
nc = xr.open_dataset(ncfile)

In [ ]:
cols = df.columns.values

In [ ]:
cols[0:-1]

In [ ]:
nc

In [ ]:
nc['depth']

In [ ]:
nc['amp']

In [ ]:
nc['amp'].plot()

In [ ]:
nc['depth'].values

In [ ]:
#nct['ADCP_CONFIG'].attrs['model_name']
nct['ADCP_CONFIG'].attrs

# TESTING/OLD

In [ ]:
fnames = sorted(os.listdir(dpath + 'JASADCP/ncfiles'))

In [ ]:
nc_counter = len(fnames)
geo_region = [None]*nc_counter

In [ ]:
ifile = 0
for fname in fnames:
    ncfile = dpath + 'JASADCP/ncfiles/' + fname
    ncnow = xr.open_dataset(ncfile)
    strnow = ncnow.attrs['cruise_sonar_summary']
    geo_regionnow = re.findall("GEOGRAPHIC_REGION *: *((?:\S+ )*\S+)", strnow)
    if geo_regionnow:
        geo_region[ifile] = geo_regionnow[0]
    ifile = ifile+1

In [ ]:
df = pd.concat(
    [pd.Series(fnames,name='fname'), pd.Series(geo_region,name='geo_region')],
    axis=1)